<a href="https://colab.research.google.com/github/hannari-python/tutorial/blob/master/trade_balance_prepro_and_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考：https://qiita.com/katsu1110/items/a8d508a1b6f07bd3a243

## Kaggle APIのための設定

最初のセルでKaggle APIトークンをgoogle colabのサーバにアップロードします。

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mazarimono001","key":"f56c0107c0f0e67d481af8dbcf141023"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install kaggle
!pip install dash jupyter_dash dash
!pip install -U plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.9.0)


## Japan trade statisticsデータのダウンロード

Kaggleのサイトにある[データ可視化の例](https://www.kaggle.com/zanjibar/year-1997-top-20-export-import-graph)を試すため、Kaggleのサイトからデータをダウンロードします。

In [ ]:
!pwd

/content


In [ ]:
!ls sample_data

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
!kaggle datasets download -d zanjibar/japan-trade-statistics

100% 697M/697M [00:14<00:00, 38.6MB/s]
100% 697M/697M [00:14<00:00, 49.6MB/s]


In [ ]:
!unzip /content/japan-trade-statistics.zip

Archive:  /content/japan-trade-statistics.zip
  inflating: codes.db                
  inflating: custom_2020.db          
  inflating: custom_from_2012.db     
  inflating: y_1997.db               
  inflating: ym_2018.db              
  inflating: ym_2019.db              
  inflating: ym_2020.db              


## データロード

# データの確認

In [ ]:
import sqlite3 
import pandas as pd 

conn = sqlite3.connect('y_1997.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master where type="table"')
for i in cursor.fetchall():
  print(i)

conn.close()

('table', 'year_from_1997', 'year_from_1997', 2, 'CREATE TABLE "year_from_1997" (\n"index" INTEGER,\n  "exp_imp" INTEGER,\n  "Year" INTEGER,\n  "Country" TEXT,\n  "Unit1" TEXT,\n  "Unit2" TEXT,\n  "Q1" INTEGER,\n  "Q2" INTEGER,\n  "Value" INTEGER,\n  "hs2" TEXT,\n  "hs4" TEXT,\n  "hs6" TEXT,\n  "hs9" TEXT\n)')


In [ ]:
conn = sqlite3.connect('codes.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master where type="table"')
for i in cursor.fetchall():
  print(i)

conn.close()

('table', 'country_jpn', 'country_jpn', 2, 'CREATE TABLE "country_jpn" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'hs2_jpn', 'hs2_jpn', 7, 'CREATE TABLE "hs2_jpn" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs2_eng', 'hs2_eng', 10, 'CREATE TABLE "hs2_eng" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs4_eng', 'hs4_eng', 77, 'CREATE TABLE "hs4_eng" (\n"hs4" TEXT,\n  "hs4_name" TEXT\n)')
('table', 'hs6_jpn', 'hs6_jpn', 118, 'CREATE TABLE "hs6_jpn" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs6_eng', 'hs6_eng', 241, 'CREATE TABLE "hs6_eng" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs9_jpn', 'hs9_jpn', 304, 'CREATE TABLE "hs9_jpn" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'hs9_eng', 'hs9_eng', 512, 'CREATE TABLE "hs9_eng" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'country_eng', 'country_eng', 685, 'CREATE TABLE "country_eng" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'hs4_jpn', 'hs4_jpn', 13

In [ ]:
def read_table(database_path, table_name):
  conn = sqlite3.connect(database_path)
  cursor = conn.cursor()
  sql = f'select * from {table_name}'
  df = pd.read_sql(sql, conn)
  conn.close()
  return df 

In [ ]:
cst_tmp = read_table('codes.db', 'custom')

In [ ]:
cst_tmp

,Custom,custom_name,pref
0,100,東京,東京
1,101,東京税関東京航空貨物出張所,東京
2,103,東京国際,東京
3,104,成田国際,東京
4,120,新潟,新潟
...,...,...,...
169,906,石垣,沖縄
170,907,那覇空港,沖縄
171,910,不開港,不開港
172,990,その他,その他


In [ ]:
data = read_table('y_1997.db', 'year_from_1997')

In [ ]:
data.shape

(6500323, 13)

In [ ]:
data

,index,exp_imp,Year,Country,Unit1,Unit2,Q1,Q2,Value,hs2,hs4,hs6,hs9
0,0,1,1997,304,None,NO,0,2,26800,01,0101,010111,010111000
1,1,1,1997,108,None,NO,0,2,180480,01,0101,010119,010119000
2,2,1,1997,207,None,NO,0,2,20000,01,0101,010119,010119000
3,3,1,1997,208,None,NO,0,2,2850,01,0101,010119,010119000
4,4,1,1997,210,None,NO,0,1,50000,01,0101,010119,010119000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6500318,231377,2,2019,245,,KG,0,128,41438,97,9706,970600,970600000
6500319,231378,2,2019,304,,KG,0,26,116584,97,9706,970600,970600000
6500320,231379,2,2019,401,,KG,0,0,7161,97,9706,970600,970600000
6500321,231380,2,2019,506,,KG,0,5,8958,97,9706,970600,970600000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6500323 entries, 0 to 6500322
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   index         int64 
 1   exp_imp       int64 
 2   Year          int64 
 3   Country       object
 4   Unit1         object
 5   Unit2         object
 6   Q1            int64 
 7   Q2            int64 
 8   Value         int64 
 9   hs2           object
 10  hs4           object
 11  hs6           object
 12  hs9           object
 13  country_name  object
dtypes: int64(6), object(8)
memory usage: 694.3+ MB


In [ ]:
country_name = read_table('codes.db', 'country_eng')
country_name = country_name[['Country', 'Country_name']]
country_name = country_name.set_index('Country')
cont_dict = country_name.to_dict()["Country_name"]
data['country_name'] = data['Country'].map(cont_dict)
data

,index,exp_imp,Year,Country,Unit1,Unit2,Q1,Q2,Value,hs2,hs4,hs6,hs9,country_name
0,0,1,1997,304,None,NO,0,2,26800,01,0101,010111,010111000,United_States_of_America
1,1,1,1997,108,None,NO,0,2,180480,01,0101,010119,010119000,Hong_Kong
2,2,1,1997,207,None,NO,0,2,20000,01,0101,010119,010119000,Netherlands
3,3,1,1997,208,None,NO,0,2,2850,01,0101,010119,010119000,Belgium
4,4,1,1997,210,None,NO,0,1,50000,01,0101,010119,010119000,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6500318,231377,2,2019,245,,KG,0,128,41438,97,9706,970600,970600000,Czech_Republic
6500319,231378,2,2019,304,,KG,0,26,116584,97,9706,970600,970600000,United_States_of_America
6500320,231379,2,2019,401,,KG,0,0,7161,97,9706,970600,970600000,Colombia
6500321,231380,2,2019,506,,KG,0,5,8958,97,9706,970600,970600000,Egypt


In [ ]:
trade_hist = data[['exp_imp', 'Year', 'Value', 'country_name']].groupby(['exp_imp', 'Year', 'country_name'], as_index=False).sum()

In [ ]:
trade_hist

,exp_imp,Year,country_name,Value
0,1,1997,Afghanistan,10636437
1,1,1997,Albania,55464
2,1,1997,Algeria,20660644
3,1,1997,American_Oceania,11133
4,1,1997,American_Samoa,1460264
...,...,...,...,...
10154,2,2019,West_Bank_and_Gaza_Strip,51222
10155,2,2019,West_Sahara,230271
10156,2,2019,Yemen,427598
10157,2,2019,Zambia,15532293


## Plotlyによる可視化
- まずは、1つの国の輸出入のグラフを作成する
- 次に複数国の輸出入のグラフを作成する

In [ ]:
|import plotly.express as px

In [ ]:
trade_hist['country_name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American_Oceania',
       'American_Samoa', 'Andorra', 'Angola', 'Antigua_and_Barbuda',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Azores_(Portugal)', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda_(UK)', 'Bhutan', 'Bolivia', 'Bosnia_and_Herzegovina',
       'Botswana', 'Brazil', 'British_Anguilla',
       'British_Antarctic_Territory', 'British_Virgin_Islands',
       'British_indian_Ocean_Territories', 'Brunei', 'Bulgaria',
       'Burkina_Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Canary_Islands_(Spain)', 'Cape_Verde', 'Cayman_islands_(UK)',
       'Central_Africa', 'Ceuta_and_Melilla_(Spain)', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook', 'Costa_Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Czech_Republic', "Côte_d'Ivoire",
       'Denmark', 'Djibouti', 'Dominica', 'Dominican_Republic', 'Ecuador',
 

In [ ]:
afgan = trade_hist[trade_hist['country_name'] == 'Afghanistan']
px.line(afgan, x='Year', y="Value", color='exp_imp')

In [ ]:
usa = trade_hist[trade_hist['country_name'] == 'United_States_of_America']
px.line(usa, x='Year', y='Value', color='exp_imp')

## exp_imp
- 多分日本からの輸出: 1
- 多分日本からの輸出: 2

In [ ]:
two_country = trade_hist[trade_hist['country_name'].isin(['China', 'Switzerland'])]

In [ ]:
two_country

,exp_imp,Year,country_name,Value
46,1,1997,China,2630720664
194,1,1997,Switzerland,256259364
270,1,1998,China,2620905420
417,1,1998,Switzerland,285325129
494,1,1999,China,2657428102
...,...,...,...,...
9681,2,2017,Switzerland,876989093
9757,2,2018,China,19193652551
9904,2,2018,Switzerland,857320750
9979,2,2019,China,18444553947


In [ ]:
px.line(two_country, x='Year', y='Value', color='exp_imp')

### Plotly Expressでは無理です

### Plotly Graph Objectsを使います

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
china = trade_hist[trade_hist['country_name'] == 'China']
china_exp = china[china['exp_imp'] == 1].reset_index(drop=True)
china_imp = china[china['exp_imp'] == 2].reset_index(drop=True)
china_balance = china_exp['Value'] - china_imp['Value']
china_balance = pd.concat([china_exp['Year'], china_balance], axis=1)

fig = make_subplots(rows=2, cols=1)


fig.add_trace(go.Scatter(x=china_exp['Year'], y=china_exp["Value"], name='輸出'), row=1, col=1)
fig.add_trace(go.Scatter(x=china_imp['Year'], y=china_imp["Value"], name='輸入'), row=1, col=1)
fig.add_trace(go.Bar(x=china_balance['Year'], y=china_balance['Value'], name='trade_balance'), row=2, col=1)

fig.update_layout(go.Layout(title=f'貿易収支(China)'))

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=1)

for cnt in ['China', 'Switzerland', 'United_States_of_America']:
  selected_df = trade_hist[trade_hist['country_name'] == cnt]
  selected_exp = selected_df[selected_df['exp_imp'] == 1].reset_index(drop=True)
  selected_imp = selected_df[selected_df['exp_imp'] == 2].reset_index(drop=True)
  trade_balance = selected_exp['Value'] - selected_imp['Value'] 
  trade_balance = pd.concat([selected_exp['Year'], trade_balance], axis=1)
  fig.add_trace(go.Scatter(x=selected_exp['Year'], y=selected_exp['Value'], name=f'{cnt}: 輸入'), row=1, col=1)
  fig.add_trace(go.Scatter(x=selected_imp['Year'], y=selected_imp['Value'], name=f'{cnt}: 輸出'), row=1, col=1)
  fig.add_trace(go.Bar(x=trade_balance['Year'], y=trade_balance['Value'], name=f'{cnt}: trade_balance'), row=2, col=1)

fig.show()

### Dashでインタラクティブに作成
- 各国の輸出、輸入を2つのグラフに分けて観察するアプリ
- 1つの国のデータをどんどん追加していくアプリ

In [ ]:
from jupyter_dash import JupyterDash 
import dash_core_components as dcc 
import dash_html_components as html 

from dash.dependencies import Input, Output

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
                       
      html.Div([
                dcc.Dropdown(id='country_selector',
                             options=[{'label': cnt, 'value': cnt} for cnt in trade_hist['country_name']],
                             value=['China'],
                             multi=True,
                             style={'width': '80%', 'display': 'inline-block'})
      ]),
      html.Div([
                
                dcc.Graph(id='exp_graph', style={'width': '48%', 'display': 'inline-block'}),
                dcc.Graph(id='imp_graph', style={'width': '48%', 'display': 'inline-block'})

      ])


])

@app.callback([Output('exp_graph', 'figure'), Output('imp_graph', 'figure')],
              [Input('country_selector', 'value')]
              )
def update_graph(selected_countries):
  selected_df = trade_hist[trade_hist['country_name'].isin(selected_countries)]
  selected_exp = selected_df[selected_df['exp_imp'] == 1]
  selected_imp = selected_df[selected_df['exp_imp'] == 2]
  fig1 = px.line(selected_exp, x='Year', y='Value', color='country_name', title='輸出')
  fig2 = px.line(selected_imp, x='Year', y='Value', color='country_name', title='輸入')
  return fig1, fig2


app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [ ]:
from dash.dependencies import State, MATCH

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div(
    [
        html.Button("PUSH ME", id="my_button", n_clicks=0),
        html.Div(id="my_div", children=[]),
    ]
)


@app.callback(
    Output("my_div", "children"),
    [Input("my_button", "n_clicks")],
    [State("my_div", "children")],
    prevent_initial_call=True,
)
def update_layout(n_clicks, exist_children):
    add_layout = html.Div(
        [
            dcc.Dropdown(
                id={"type": "my_dropdown", "index": n_clicks},
                options=[
                    {"value": cnt, "label": cnt}
                    for cnt in trade_hist["country_name"].unique()
                ],
                value=trade_hist["country_name"].unique()[n_clicks - 1],
            ),
            dcc.Graph(id={"type": "add_graph", "index": n_clicks}),
        ], style={'width': '23%', 'display': 'inline-block', 'margin': '1%'}
    )
    exist_children.append(add_layout)
    return exist_children


@app.callback(
    Output({"type": "add_graph", "index": MATCH}, "figure"),
    [Input({"type": "my_dropdown", "index": MATCH}, "value")],
)
def update_graph(selected_country):
    selected_df = trade_hist[trade_hist["country_name"] == selected_country]
    return px.line(
        selected_df,
        x="Year",
        y="Value",
        color="exp_imp",
        title=f"{selected_country}の輸出入額（1:輸出, 2:輸入）",
    )

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

In [ ]:
trade_hist.to_csv('trade_hist.csv')

In [ ]:
import sqlite3 
import pandas as pd 

conn = sqlite3.connect('./y_1997.db')
cursor = conn.cursor()
sql = 'select * from year_from_1997'
df = pd.read_sql(sql, conn)

In [ ]:
conn.close()
df.head()

In [ ]:
conn = sqlite3.connect('/content/codes.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master')


for t in cursor.fetchall():
  print(t)

conn.close()

('table', 'country_jpn', 'country_jpn', 2, 'CREATE TABLE "country_jpn" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'hs2_jpn', 'hs2_jpn', 7, 'CREATE TABLE "hs2_jpn" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs2_eng', 'hs2_eng', 10, 'CREATE TABLE "hs2_eng" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs4_eng', 'hs4_eng', 77, 'CREATE TABLE "hs4_eng" (\n"hs4" TEXT,\n  "hs4_name" TEXT\n)')
('table', 'hs6_jpn', 'hs6_jpn', 118, 'CREATE TABLE "hs6_jpn" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs6_eng', 'hs6_eng', 241, 'CREATE TABLE "hs6_eng" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs9_jpn', 'hs9_jpn', 304, 'CREATE TABLE "hs9_jpn" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'hs9_eng', 'hs9_eng', 512, 'CREATE TABLE "hs9_eng" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'country_eng', 'country_eng', 685, 'CREATE TABLE "country_eng" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'hs4_jpn', 'hs4_jpn', 13

In [ ]:
conn = sqlite3.connect('./trade_meta_data.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master')

for i in cursor:
  print(i)
conn.close()

('table', 'country_jpn', 'country_jpn', 2, 'CREATE TABLE "country_jpn" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'hs2_jpn', 'hs2_jpn', 7, 'CREATE TABLE "hs2_jpn" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs2_eng', 'hs2_eng', 10, 'CREATE TABLE "hs2_eng" (\n"hs2" TEXT,\n  "hs2_name" TEXT\n)')
('table', 'hs4_eng', 'hs4_eng', 77, 'CREATE TABLE "hs4_eng" (\n"hs4" TEXT,\n  "hs4_name" TEXT\n)')
('table', 'hs6_jpn', 'hs6_jpn', 118, 'CREATE TABLE "hs6_jpn" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs6_eng', 'hs6_eng', 241, 'CREATE TABLE "hs6_eng" (\n"hs6" TEXT,\n  "hs6_name" TEXT\n)')
('table', 'hs9_jpn', 'hs9_jpn', 304, 'CREATE TABLE "hs9_jpn" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'hs9_eng', 'hs9_eng', 512, 'CREATE TABLE "hs9_eng" (\n"hs9" TEXT,\n  "hs9_name" TEXT\n)')
('table', 'country_eng', 'country_eng', 685, 'CREATE TABLE "country_eng" (\n"Country" TEXT,\n  "Country_name" TEXT,\n  "Area" TEXT\n)')
('table', 'custom', 'custom', 6, '

In [ ]:
conn = sqlite3.connect('./trade_meta_data.db')
cursor = conn.cursor()
sql = 'select * from hs2_jpn'

df_hs2 = pd.read_sql(sql, conn)

conn.close()

In [ ]:
df_hs2.head()

,hs2,hs2_name
0,01,動物（生きているものに限る。）
1,02,くず肉
2,03,魚 エビ、カニ、タコ、貝類など
3,04,酪農品、鳥卵、天然はちみつ及びその他（他の分類でないもの）
4,05,動物性生産品（他の類にないもの）


In [ ]:
conn = sqlite3.connect('./trade_meta_data.db')
cursor = conn.cursor()
sql = 'select * from custom'

df_custom = pd.read_sql(sql, conn)

conn.close()

df_custom.head()

,Custom,custom_name
0,100,東京
1,101,東京税関東京航空貨物出張所
2,103,東京国際
3,104,成田国際
4,120,新潟


In [ ]:
conn = sqlite3.connect('./y_1997.db')
cursor = conn.cursor()
cursor.execute('select * from sqlite_master where type="table"')

for i in cursor:
  print(i)

conn.close()

('table', 'year_from_1997', 'year_from_1997', 2, 'CREATE TABLE "year_from_1997" (\n"index" INTEGER,\n  "exp_imp" INTEGER,\n  "Year" INTEGER,\n  "Country" TEXT,\n  "Unit1" TEXT,\n  "Unit2" TEXT,\n  "Q1" INTEGER,\n  "Q2" INTEGER,\n  "Value" INTEGER,\n  "hs2" TEXT,\n  "hs4" TEXT,\n  "hs6" TEXT,\n  "hs9" TEXT\n)')


In [ ]:
sql="""
select exp_imp,Year,sum(Value) as Value, Country from year_from_1997
group by Year, Country, exp_imp
"""[1:-1]
dff = pd.read_sql(sql,conn)
dff

,exp_imp,Year,Value,Country
0,1,1997,3153238334,103
1,2,1997,1762757028,103
2,1,1997,21630387,104
3,2,1997,36535244,104
4,1,1997,2630720664,105
...,...,...,...,...
10217,1,2019,5526581,627
10218,2,2019,8278,627
10219,1,2019,2010416,628
10220,2,2019,1710270,628


In [ ]:
country_eng_df.head()

,Country,Country_name,Area
0,103,Korea,Asia
1,104,North_Korea,Asia
2,105,China,Asia
3,106,Taiwan,Asia
4,107,Mongolia,Asia


# Plotly.Expressを使った可視化

In [ ]:
import plotly.express as px

In [ ]:
def show_plotly_graph(country_code, df, df_cnt):
  xdf = df[df["Country"] == country_code]
  country_name = df_cnt[df_cnt["Country"] == country_code]["Country_name"].values[0]
  return px.line(xdf, x="Year", y="Value", color="exp_imp", title=country_name)

In [ ]:
show_plotly_graph('106', dff, country_eng_df)

# JupyterDashによるアプリケーション化

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc 
import dash_html_components as html 
from dash.dependencies import Input, Output 

In [ ]:
for num, test in enumerate(country_eng_df):
  print(num, test)

0 Country
1 Country_name
2 Area


In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
           dcc.Dropdown(id='country_dropdown',
                        options=[{'label': country_eng_df.iloc[num, 1], 'value': country_eng_df.iloc[num, 0]} for num in range(len(country_eng_df))],
                        value='103'
                        ),
            dcc.Graph(id="selected_graph")        
          
])

@app.callback(Output('selected_graph', 'figure'), [Input('country_dropdown', 'value')])
def update_graph(selected_value):
  return show_plotly_graph(selected_value, dff, country_eng_df)

app.run_server(mode='inline')

<IPython.core.display.Javascript object>